<h1>
<center>
Module 7 - A real world problem
</center>
</h1>
<div class=h1_cell>
<p>
We will be back to doing prediction. The problem is that the site donorschoose.org is overwhelmed with applications. You can read the details at https://www.kaggle.com/c/donorschoose-application-screening (yes, this is a Kaggle competition). Briefly, teachers apply for funding to help them further their classroom teaching. The application has a number of boxes to fill in. We will concentrate on just three: the first essay they must write, the second essay they must write and the resources they need for their project. Our target column is 'project_is_funded', a binary column with 0 meaning their application was rejected. Here is more background:
<p>
  <img src='https://www.dropbox.com/s/ka7k4in6dcmojrw/Screenshot%202019-05-02%2013.51.24.png?raw=1'>
  <p>

  <p>
First, let's bring in our library and the data.



In [0]:
#flush the old directory
!rm -r  'w20_ds_library'

In [0]:
my_github_name = 'FutureDeus'  #replace with your account name

In [0]:
clone_url = f'https://github.com/{my_github_name}/w20_ds_library.git'

In [4]:
#get the latest.
!git clone $clone_url 


Cloning into 'w20_ds_library'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 93 (delta 58), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (93/93), done.


In [5]:
from w20_ds_library import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import pandas as pd
url = 'https://www.dropbox.com/s/ofjalkxqs0rtb97/train.csv?raw=1'
donate_table = pd.read_csv(url, encoding='utf-8')

In [7]:
len(donate_table)  #it's a big one!

182080

In [8]:
donate_table.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


So what we have our 3 primary text columns. I am ignorning project eassays 3 and 4 because they have substantial empties. I propose we use eassays 1 and 2 and the resource description to build a bag of words that can be used by `robust_bayes`.

##Do some wrangling first

Rename target column as "label".

In [0]:
donate_table = donate_table.rename(columns={'project_is_approved': 'label'})

##What do we have to beat?

In [10]:
donate_table['label'].tolist().count(1)/len(donate_table)  #0.8476823374340949

0.8476823374340949

##So 85% are accepted

If we cannot do better than 85%, then what is the point?

##Split into training and testing

In [0]:
import numpy as np
rsgen = np.random.RandomState(1234)  #use this so we get same values

In [0]:
donate_table = donate_table.sample(frac=1, axis=1, random_state=rsgen).reset_index(drop=True)


split things out

In [13]:
split_n = int(len(donate_table)/3)
split_n

60693

In [0]:
testing_table = donate_table[:split_n].reset_index(drop=True)
training_table = donate_table[split_n:].reset_index(drop=True)

In [15]:
testing_table.head()

,project_resource_summary,project_essay_3,id,project_essay_4,teacher_number_of_previously_posted_projects,teacher_prefix,project_essay_2,project_subject_subcategories,teacher_id,project_essay_1,project_title,project_submitted_datetime,project_grade_category,project_subject_categories,school_state,label
0,My students need 6 Ipod Nano's to create and d...,NaN,p036502,NaN,26,Ms.,I currently have a differentiated sight word c...,Literacy,484aaf11257089a66cfedc9461c6bd0a,Most of my kindergarten students come from low...,Super Sight Word Centers,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,NV,1
1,My students need matching shirts to wear for d...,NaN,p039565,NaN,1,Mrs.,We strive to provide our diverse population of...,"Performing Arts, Team Sports",df72a3ba8089423fa8a94be88060f6ed,Our elementary school is a culturally rich sch...,Keep Calm and Dance On,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports",GA,0
2,My students need the 3doodler. We are an SEM s...,NaN,p233823,NaN,5,Ms.,We are looking to add some 3Doodler to our cla...,"Applied Sciences, Literature & Writing",a9b876a9252e08a55e3d894150f75ba3,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,Lets 3Doodle to Learn,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language",UT,1
3,My students need balls and other activity equi...,NaN,p185307,NaN,16,Mr.,"The student's project which is totally \""kid-i...",Health & Wellness,525fdbb6ec7f538a48beebaa0a51b24f,My students are the greatest students but are ...,"\""Kid Inspired\"" Equipment to Increase Activit...",2016-08-12 15:42:11,Grades 3-5,Health & Sports,NC,0
4,My students need a water filtration system for...,NaN,p013780,NaN,42,Mr.,For some reason in our kitchen the water comes...,Health & Wellness,a63b5547a7239eae4c1872670848e61a,My students are athletes and students who are ...,We need clean water for our culinary arts class!,2016-08-06 09:09:11,Grades 6-8,Health & Sports,CA,1


<h2>
Challenge 1 - Build the word bag
</h2>

I'd like you to take as your text the concatenation of three columns: '`project_essay_1`',
'`project_essay_2`', and '`project_resource_summary`'. This will give you one large string per row. Use that string as your raw text to `get_clean_words`. Then build up your bag of words from that. Remember to only count unique words per applicant.

This took me about 15 minutes of compute time. I'll show you how to write out your bag so you don't have to build it more than once.
 

In [16]:
%%time
template = [[1, 0], [0, 1]]
bag_of_words = {}
for index, row in training_table.iterrows():
  words = set(get_clean_words(swords, row['project_essay_1'] + row['project_essay_2'] + row['project_resource_summary']))
  for i in words:
    if i in bag_of_words:
      bag_of_words[i][row['label']] += 1
    else:
      bag_of_words[i] = list(template[row['label']])

CPU times: user 12min 48s, sys: 170 ms, total: 12min 48s
Wall time: 12min 48s


In [17]:
#match against my results below

sorted(bag_of_words.items())[:30]

[('0', [37, 291]),
 ('00', [51, 196]),
 ('000', [111, 805]),
 ('0000blk', [0, 1]),
 ('000s', [1, 0]),
 ('000square', [0, 1]),
 ('001', [0, 2]),
 ('002', [1, 0]),
 ('00am', [4, 10]),
 ('00p', [0, 1]),
 ('00pm', [0, 14]),
 ('01', [1, 10]),
 ('010', [0, 1]),
 ('01075rm', [0, 1]),
 ('0125', [0, 2]),
 ('013', [0, 1]),
 ('015', [0, 1]),
 ('016011r', [0, 1]),
 ('016g', [0, 1]),
 ('01g', [0, 1]),
 ('01ip', [0, 1]),
 ('02', [1, 3]),
 ('021', [0, 2]),
 ('022x10', [0, 1]),
 ('023', [0, 1]),
 ('025', [0, 1]),
 ('02810010', [0, 1]),
 ('03', [1, 6]),
 ('030', [1, 0]),
 ('03074', [0, 1])]

<pre>
[('0', [37, 291]),
 ('00', [51, 196]),
 ('000', [111, 805]),
 ('0000blk', [0, 1]),
 ('000s', [1, 0]),
 ('000square', [0, 1]),
 ('001', [0, 2]),
 ('002', [1, 0]),
 ('00am', [4, 10]),
 ('00p', [0, 1]),
 ('00pm', [0, 14]),
 ('01', [1, 10]),
 ('010', [0, 1]),
 ('01075rm', [0, 1]),
 ('0125', [0, 2]),
 ('013', [0, 1]),
 ('015', [0, 1]),
 ('016011r', [0, 1]),
 ('016g', [0, 1]),
 ('01g', [0, 1]),
 ('01ip', [0, 1]),
 ('02', [1, 3]),
 ('021', [0, 2]),
 ('022x10', [0, 1]),
 ('023', [0, 1]),
 ('025', [0, 1]),
 ('02810010', [0, 1]),
 ('03', [1, 6]),
 ('030', [1, 0]),
 ('03074', [0, 1])]
 </pre>

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json

Note that you may need to change your path. I have a folder called class_tables. If you do not, wrangle the string into shape for you.

In [0]:
#write out bag
with open('/content/gdrive/My Drive/class_tables/bag_of_words.json', 'w') as file:
    file.write(json.dumps(bag_of_words))

In [0]:
#read in bag

with open('/content/gdrive/My Drive/class_tables/bag_of_words.json', 'r') as file:
    bag_of_words = json.load(file)

In [22]:
#should match above
sorted(bag_of_words.items())[:30]

[('0', [37, 291]),
 ('00', [51, 196]),
 ('000', [111, 805]),
 ('0000blk', [0, 1]),
 ('000s', [1, 0]),
 ('000square', [0, 1]),
 ('001', [0, 2]),
 ('002', [1, 0]),
 ('00am', [4, 10]),
 ('00p', [0, 1]),
 ('00pm', [0, 14]),
 ('01', [1, 10]),
 ('010', [0, 1]),
 ('01075rm', [0, 1]),
 ('0125', [0, 2]),
 ('013', [0, 1]),
 ('015', [0, 1]),
 ('016011r', [0, 1]),
 ('016g', [0, 1]),
 ('01g', [0, 1]),
 ('01ip', [0, 1]),
 ('02', [1, 3]),
 ('021', [0, 2]),
 ('022x10', [0, 1]),
 ('023', [0, 1]),
 ('025', [0, 1]),
 ('02810010', [0, 1]),
 ('03', [1, 6]),
 ('030', [1, 0]),
 ('03074', [0, 1])]

##We will be using robust_bayes

So make sure that is in your library.

###A small test

In [23]:
row0_text = testing_table.loc[0, 'project_essay_1'] + testing_table.loc[0, 'project_essay_2'] + testing_table.loc[0, 'project_resource_summary']
row0_text

'Most of my kindergarten students come from low-income households and are considered \\"at-risk\\". These kids walk to school alongside their parents and most have never been further than walking distance from their house. For 80% of my students, English is not their first language or the language spoken at home. \\r\\n\\r\\nWhile my kindergarten kids have many obstacles in front of them, they come to school each day excited and ready to learn. Most students started the year out never being in a school setting. At the start of the year many had never been exposed to letters. Each day they soak up more knowledge and try their hardest to succeed. They are highly motivated to learn new things every day. We are halfway through the year and they are starting to take off. They know know all letters, some sight words, numbers to 20, and a majority of their letter sounds because of their hard work and determination. I am excited to see the places we will go from here!I currently have a differe

In [24]:
%time robust_bayes(set(get_clean_words(swords, row0_text)), bag_of_words, training_table)  #(2.280283838979279e-215, 5.454034996032865e-162)


CPU times: user 15.2 ms, sys: 0 ns, total: 15.2 ms
Wall time: 15.1 ms


(2.280283838979279e-215, 5.454034996032862e-162)

#Challenge 2

I'll ask you to write a new function, `efficient_bayes_tester`, using `bayes_tester`  from your library as a starting point. Here is the new function signature line:
<pre>
def efficient_bayes_tester(testing_list:list, evidence_bag:dict, training_table:dframe, parser:Callable) -> list:
</pre>
So the new tester no longer takes the `testing_table` as input. Instead, it takes a list of raw strings, one per row in testing table. So the first thing you have to do is build up the testing list. Go ahead and do that now.

In [25]:
%time
testing_list = [row['project_essay_1'] + row['project_essay_2'] + row['project_resource_summary'] for key, row in testing_table.iterrows()]


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [26]:
testing_list[0] == row0_text  #True

True

###Make sure you match my results below

<pre>
Most of my kindergar
Our elementary schoo
Hello;\r\nMy name is
My students are the 
My students are athl
</pre>

In [27]:
for i in range(5):
  print(testing_list[i][:20])

Most of my kindergar
Our elementary schoo
Hello;\r\nMy name is
My students are the 
My students are athl


#Challenge 3

Define your new function `efficient_bayes_tester`. I copied on `bayes_tester` then made changes to it.

In [0]:
def efficient_bayes_tester(testing_list:list, evidence_bag:dict, training_table:dframe, parser:Callable) -> list:
  assert isinstance(testing_list, list), f'testing_list is not a list but instead a {type(testing_list)}'
  assert isinstance(evidence_bag, dict), f'evidence_bag not a dict but instead a {type(evidence_bag)}'
  assert isinstance(training_table, pd.core.frame.DataFrame), f'training_table not a dataframe but instead a {type(training_table)}'
  assert callable(parser), f'parser not a function but instead a {type(parser)}'

  #your code here
  return [robust_bayes(set(parser(i)), evidence_bag, training_table) for i in testing_list]


##Curry get_clean_words

In [0]:
def curry_x(old_func,x):
  #the body of curry_x is the closure. It contains the value of old_func and x.

  def inner(y): return old_func(x,y)  #define a new function that uses x from closure and passes in y.

  return inner  #will return a function of 1 argument. But that function will carry the closure with it.

In [0]:
#define word_parser here

word_parser = curry_x(get_clean_words, swords)

##A small test to check

In [31]:
%time triples = efficient_bayes_tester(testing_list[:100], bag_of_words, training_table, word_parser)  #First 100

CPU times: user 1.04 s, sys: 2 ms, total: 1.04 s
Wall time: 1.05 s


In [32]:
triples[:10]

[(2.280283838979279e-215, 5.454034996032862e-162),
 (3.2722035943058405e-225, 8.835721453823339e-178),
 (2.023331626063151e-185, 2.494149804383589e-143),
 (1.873383589926185e-213, 1.2099928342454241e-163),
 (4.7530522355398075e-158, 1.971334175054714e-129),
 (3.8371130318262005e-228, 9.761914772604384e-179),
 (1.123338075166983e-222, 3.751113068045983e-169),
 (2.269125777988007e-136, 5.633266965875088e-98),
 (7.107712813614198e-198, 1.2701621290961848e-148),
 (1.0146527724793372e-189, 8.271947458182652e-146)]

<pre>
[(2.280283838979279e-215, 5.454034996032865e-162),
 (3.272203594305844e-225, 8.835721453823339e-178),
 (2.0233316260631495e-185, 2.4941498043835897e-143),
 (1.8733835899261855e-213, 1.2099928342454242e-163),
 (4.753052235539806e-158, 1.971334175054715e-129),
 (3.8371130318261993e-228, 9.76191477260439e-179),
 (1.1233380751669828e-222, 3.751113068045984e-169),
 (2.2691257779880076e-136, 5.6332669658750894e-98),
 (7.107712813614193e-198, 1.2701621290961858e-148),
 (1.0146527724793382e-189, 8.271947458182665e-146)]
 </pre>

###Now for the whole table - 14 minutes for me

In [33]:
%time triples = efficient_bayes_tester(testing_list, bag_of_words, training_table, word_parser)

CPU times: user 10min 41s, sys: 1.44 s, total: 10min 42s
Wall time: 10min 43s


In [34]:
triples[-10:]

[(2.7516386028309955e-150, 1.7053794755123355e-111),
 (4.714017675504303e-296, 7.301645981382889e-236),
 (6.395879048033359e-200, 5.810783725353376e-153),
 (2.2250738585072014e-308, 7.418372334071461e-275),
 (1.0152991467628835e-147, 2.603277120848649e-113),
 (2.211989061228608e-151, 5.656249756712308e-115),
 (3.9214382282675495e-132, 1.1387629630568396e-96),
 (1.5884697154516175e-229, 2.5111578436915938e-171),
 (5.421184230172825e-206, 4.3084454590373585e-150),
 (5.44774437666831e-307, 9.236725142410577e-237)]

<pre>
[(2.751638602830995e-150, 1.7053794755123348e-111),
 (4.7140176755043113e-296, 7.301645981382898e-236),
 (6.395879048033355e-200, 5.810783725353377e-153),
 (2.2250738585072014e-308, 7.418372334071456e-275),
 (1.0152991467628838e-147, 2.60327712084865e-113),
 (2.2119890612286062e-151, 5.656249756712314e-115),
 (3.921438228267547e-132, 1.1387629630568394e-96),
 (1.5884697154516164e-229, 2.511157843691593e-171),
 (5.421184230172821e-206, 4.308445459037365e-150),
 (5.447744376668315e-307, 9.236725142410574e-237)]
</pre>

#Challenge 4

See how you did. I got an accuracy of 0.8379549536190335.

In [0]:
#your code
cm = {
    (0, 0): 0 ,
    (0, 1): 0 ,
    (1, 0): 0 ,
    (1, 1): 0
}
actuals = testing_table.label.to_list()
for i in range(len(triples)):
  if triples[i][1] > triples[i][0]:
    cm[(1 , label_list[i])] += 1
  else:
    cm[(0 , label_list[i])] += 1

In [41]:
cm  #{(0, 0): 133, (0, 1): 847, (1, 0): 8988, (1, 1): 50725}

{(0, 0): 133, (0, 1): 847, (1, 0): 8988, (1, 1): 50725}

In [42]:
(133+50725)/len(actuals)  #0.8379549536190335

0.8379549536190335

#What to make of this

We were trying to beat 0.8497190779826339. We came close. But spent a lot of time to just produce a result we could have gotten by choosing a constant 1.

Bottomline: NB, at least as we have set it up, is not a winner for this problem. This is not totally unexpected. You generally have to play with different modeling tools to get the right one. It looks like NB is not the right one for this. And I can tell you KNN will be a disaster with such a large table even with our fast clustering version. Think about the size of the vector. It will be in 65 thousand dimensions unless we do something to reduce it.

That is where I would like to head next. Introduce a new model called Artifical Neural Nets (ANN). But before that, look at new ways of transforming text on a row to a single 300-item vector. We can feed these vectors to an ANN in a strightforward fashion.